Build a GA that constructs exam or class timetables under realistic constraints.
For the project, choose one small or medium-sized instance (from Toronto or ITC 2007) and
implement at least:
● all hard constraints (no student in two exams at the same time),
● one or two soft criteria in your fitness function.

By 10 December you should submit a complete mini-project:
A working GA implementation for your chosen problem and dataset.

- At least a few experiments:  
   -  different parameter settings and/or different fitness/operators,  
   -  several runs per configuration, with basic metrics (best/average quality, etc.).  

- clean repository:  
   - readable code  
   - clear instructions in README (how to run, what to expect),  
   - a short summary of results and main conclusions.  

I'm going to use Toronto dataset

In [2]:
import random
from typing import List, Tuple

# For reproducibility
random.seed(42)

test_dataset = "datasets/ear-f-83-2"
exam_dataset_path = test_dataset + ".crs"
students_dataset_path = test_dataset + ".stu"

Representation format: Array of N elements, where N - number of exams.  
So genom always contains all and indeed valid.

In [3]:
my_candidate = [0, 1, 2, 0, 3, 2]

In [4]:
def read_exams(path: str) -> List[int]:
    f = open(path, "r")
    frl = f.readlines()
    exams = [int(el.split()[0]) for el in frl]
    return exams

def read_students(path: str) -> List[List[int]]:
    f = open(path, "r")
    frl = f.readlines()
    students = list(map(lambda x: list(map(int, x.split())), frl))
    return students

exams = read_exams(exam_dataset_path)
students = read_students(students_dataset_path)

number_of_exams = len(exams)
number_of_students = len(students)

c = [[0 for _ in range(number_of_exams + 1)] for _ in range(number_of_exams + 1)]

def intersect_exams(students: List[List[int]])-> List[List[int]]:
    for student in students:
        for a in student:
            for b in student:
                if a != b:
                    c[a][b] += 1

intersect_exams(students)

In [24]:
from math import comb

def fitness(individual: List[int]) -> int:
    """
    Compute fitness for an exam configuration.

    Requirement:
    - higher is better;
    """
    exams = dict()
    max_fitness = 0
    fit = max_fitness
    for i in range(number_of_exams):
        exams.setdefault(individual[i], [])
        exams[individual[i]].append(i)
    for key1, value1 in exams.items():
        for key2, value2 in exams.items():
            if key1 >= key2:
                continue
            for i in range(len(value1)):
                for j in range(len(value2)):
                    fit -= (2 ** (4 - abs(key1 - key2))) * c[value1[i]][value2[j]]
        

    return fit

In [25]:
my_candidate = list(random.randint(0, 5) for _ in range(number_of_exams))
fit_value = fitness(my_candidate)
print(f"Fitness of candidate {my_candidate} is {fit_value}")

Fitness of candidate [3, 0, 3, 2, 2, 5, 3, 4, 0, 4, 5, 3, 0, 0, 3, 0, 4, 3, 2, 5, 5, 4, 1, 4, 3, 5, 5, 5, 3, 3, 5, 4, 5, 1, 5, 1, 3, 2, 5, 5, 3, 5, 4, 5, 2, 4, 1, 2, 1, 3, 4, 0, 5, 5, 2, 4, 0, 0, 2, 4, 4, 3, 2, 1, 0, 1, 4, 4, 3, 4, 5, 5, 2, 5, 3, 0, 4, 3, 1, 2, 2, 5, 2, 4, 3, 0, 3, 5, 0, 4, 3, 2, 2, 3, 0, 5, 1, 4, 2, 1, 0, 3, 0, 1, 2, 4, 4, 0, 0, 3, 2, 3, 5, 0, 0, 0, 1, 4, 2, 2, 5, 5, 2, 1, 4, 0, 5, 2, 4, 0, 2, 4, 3, 3, 1, 0, 2, 3, 4, 1, 0, 0, 2, 3, 2, 3, 2, 2, 0, 0, 4, 1, 5, 5, 4, 5, 5, 3, 4, 0, 0, 4, 5, 0, 0, 1, 3, 3, 1, 5, 3, 1, 4, 1, 0, 0, 1, 0, 1, 4, 0, 2, 0, 4, 2, 3, 4, 5, 2] is -97591.5


### Tournament

In [26]:
def create_initial_population(pop_size: int,
                              exam_count: int,
                              max_time_slot: int = 50) -> List[List[int]]:
    """Create an initial population of random individuals."""
    population = []
    for _ in range(pop_size):
        individual = [random.randint(0, max_time_slot - 1)
                      for _ in range(exam_count)]
        population.append(individual)
    return population

def tournament_selection(population: List[List[int]],
                         fitnesses: List[int],
                         k: int = 3) -> List[int]:
    """Select one parent using tournament selection."""
    idxs = random.sample(list(range(len(population))), k)
    fitnesses = [(fitnesses[i], i) for i in idxs]
    return population[max(fitnesses)[1]]
population = create_initial_population(10, number_of_exams)

### Crossover

In [27]:
from typing import Tuple, List
import random

def one_point_crossover(parent1: List[int],
                        parent2: List[int],
                        crossover_prob: float = 0.8) -> Tuple[List[int], List[int]]:
    """
    Perform one-point crossover with given probability.
    """
    n = number_of_exams

    r = random.random()
    if r > crossover_prob:
      return (parent1, parent2)
    c = random.randint(1, n-2)
    child1 = parent1[:c] + parent2[c:]
    child2 = parent2[:c] + parent1[c:]

    return (child1, child2)

### Mutation

In [28]:
def mutate(individual: List[int],
           mutation_prob: float = 0.1) -> None:
    """
    Mutate the individual *in place*.
    """
    n = number_of_exams
    for i in range(n):
      if random.random() < mutation_prob:
        individual[i] = random.randint(0, n-1)

### Main loop

In [29]:
def genetic_algorithm(pop_size: int = 50,
                      max_generations: int = 200,
                      crossover_prob: float = 0.8,
                      mutation_prob: float = 0.1,
                      tournament_k: int = 3,
                      verbose: bool = True):
    """
    Run the genetic algorithm and return:
    - best individual found,
    - its fitness,
    - generation index when the optimum was first reached (or None if not reached).
    """
    # 1) Initialise population
    population = create_initial_population(pop_size, number_of_exams)

    best_individual = None
    best_fitness = float("-inf")
    success_generation = None  # will stay None if we never hit the optimum

    for gen in range(max_generations):
        fitnesses = [fitness(ind) for ind in population]

        best_idx = max(range(len(population)), key=lambda i: fitnesses[i])
        if fitnesses[best_idx] > best_fitness:
            best_fitness = fitnesses[best_idx]
            best_individual = population[best_idx][:]

        if verbose and gen % 10 == 0:
            print(f"Generation {gen:3d}: best fitness = {best_fitness}")

        max_possible = 0
        if best_fitness == max_possible:
            success_generation = gen
            if verbose:
                print(f"Solution found at generation {gen}!")
            break

        # 5) Create new population
        new_population: List[List[int]] = []

        while len(new_population) < pop_size:
            parent1 = tournament_selection(population, fitnesses, tournament_k)
            parent2 = tournament_selection(population, fitnesses, tournament_k)
            children = one_point_crossover(parent1, parent2, crossover_prob)
            mutate(children[0], mutation_prob)
            mutate(children[1], mutation_prob)
            new_population.append(children[0])
            new_population.append(children[1])

        population = new_population[:pop_size]

    return best_individual, best_fitness, success_generation

best, best_f, gen_succ = genetic_algorithm()
print("Best fitness:", best_f)
print("Success generation:", gen_succ)


Generation   0: best fitness = -13212.698223898211
Generation  10: best fitness = -2478.5571090002495
Generation  20: best fitness = -2298.4399701688676
Generation  30: best fitness = -2298.4399701688676
Generation  40: best fitness = -2289.398138566178
Generation  50: best fitness = -2201.771174636039
Generation  60: best fitness = -2168.8954289474354
Generation  70: best fitness = -2168.8954289474354
Generation  80: best fitness = -2168.8954289474354
Generation  90: best fitness = -2168.8954289474354
Generation 100: best fitness = -2168.8954289474354
Generation 110: best fitness = -2168.8954289474354
Generation 120: best fitness = -2168.8954289474354
Generation 130: best fitness = -2168.8954289474354
Generation 140: best fitness = -2164.8624089450354
Generation 150: best fitness = -2164.8624089450354
Generation 160: best fitness = -2164.8624089450354
Generation 170: best fitness = -2164.8624089450354
Generation 180: best fitness = -1999.3759151269935
Generation 190: best fitness = -1